# Day 21: Springdroid Adventure

https://adventofcode.com/2019/day/21#part2

## Part 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from AOC2019 import Intcode, IntcodeV2, readIntcodeProg

In [2]:
prog = readIntcodeProg("input21.txt")

Two registers are available: 
- T, the temporary value register
- J, the jump register. 

If the jump register is true at the end of the springscript program, the springdroid will try to jump. Both of these registers start with the value false.

The springdroid can detect ground at four distances: 

- one tile away (A), 
- two tiles away (B), 
- three tiles away (C), 
- four tiles away (D). 

If there is ground at the given distance, the register will be true; if there is a hole, the register will be false.

There are only three instructions available in springscript:

- AND X Y sets Y to true if both X and Y are true; otherwise, it sets Y to false.
- OR X Y sets Y to true if at least one of X or Y is true; otherwise, it sets Y to false.
- NOT X Y sets Y to true if X is false; otherwise, it sets Y to false.

In all three instructions, the second argument (Y) needs to be a writable register (either T or J). The first argument (X) can be any register (including A, B, C, or D).

In [ ]:
def runSpringcode(prog,commands):
    # convert commands to ASCII
    com_ascii = []
    for l in commands:
        for c in l:
            com_ascii.append(ord(c))
        com_ascii.append(10)
    # initialize and run Incode
    d = IntcodeV2(prog,"DROID",com_ascii)
    d.reset()
    d.runProgram()
    #convert output to ASCII
    o = d.getOutput()
    output = []
    line = ""
    hulldamage = -1
    for i in o:
        if (i>127):
            hulldamage = i
            print("Found hull damage = ",hulldamage)
            return output, hulldamage
        if (i!=10):
            line += chr(i)
        else:
            line += '\n'
            output.append(line)
            line = ""
    return output, hulldamage

In [126]:
commands = [
    'NOT D J',
    'WALK']

commands = [
    'NOT A T',
    'OR T J',
    'AND D J',
    'WALK' ]

# this is the first solution found by brute force:
#commands = ['NOT A T', 'NOT C J', 'AND D J', 'OR T J', 'WALK']

output, hulldamage = runSpringcode(prog,commands)

print("Hull damage =", hulldamage,"\n")
for l in output:
    print(l,end="")

Hull damage = -1 

Input instructions:

Walking...


Didn't make it across:

.................
.................
@................
#####.#..########

.................
.................
.@...............
#####.#..########

.................
.................
..@..............
#####.#..########

.................
.................
...@.............
#####.#..########

.................
.................
....@............
#####.#..########

.................
.................
.................
#####@#..########



I can attempt to brute force the solution by feeding the program all possible permutations of instructions. Of course the longer the successfull program is, the longer it'll take to find it... A solution is found with 4 instructions, but it took quite some time!

The search space can probably be reduced by making some assumption given the result of the first tests. For instance, given the nature of a jump  (see above) the springdroid should jump when the first sensor (A) is false, and should NOT jump when the fourth sensor is false (like it is doing in the above example), so any successful program should end with these 3 instructions:

NOT A T

OR T J

AND D J

This also means that the program should at least be 4 instruction long, since these 3 instructions alone fail.

In [ ]:
# Any instruction can be composed as [AND,OR,NOT] x [A,B,C,D,T,J] x [T,J]

bools = ["AND","OR","NOT"] # 3
tiles = ["A","B","C","D","T","J"] # 6
regs = ["T","J"] # 2

instr = []
for b in bools:
    for t in tiles:
        for r in regs:
            c = b+' '+t+' '+r
            instr.append(c)
            
#print(instr)

In [133]:
from itertools import permutations

commands = []
output = []

found = False
imin = 1
imax = 4

# Full brute force scan
#for n in range(imin,imax+1):
#    print("Testing all possible sequences with",n,"instructions...")
#    perm = permutations(instr,n)
#    for p in perm:
#        commands = []
#        for i in p:
#            commands.append(i)
#        commands.append('WALK')
#        output, hulldamage = runSpringcode(prog,commands)
#        if (hulldamage != -1):
#            print("Found solution!")
#            print("Instructions =",commands)
#            found = True
#            break
#    if found:
#        break

# slightly more clever scan

endcoms = [
    'NOT A T',
    'OR T J',
    'AND D J' ]

found = False
imin = 4
imax = 10

for n in range(imin,imax+1):
    print("Testing all possible sequences with",n,"instructions...")
    perm = permutations(instr,n-len(endcoms))
    for p in perm:
        commands = []
        for i in p:
            commands.append(i)
        for i in endcoms: # add ending command sequence
            commands.append(i)
        commands.append('WALK')
        ##print(commands)
        output, hulldamage = runSpringcode(prog,commands)
        if (hulldamage != -1):
            print("Found solution! Instructions =",commands)
            found = True
            break
    if found:
        break

Testing all possible sequences with 4 instructions...
Found hull damage =  19357180
Found solution! Instructions = ['NOT C J', 'NOT A T', 'OR T J', 'AND D J', 'WALK']
